In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn import svm
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn import ensemble
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import SMOTE
from pandas_ml import ConfusionMatrix
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
%matplotlib inline


C:\Users\UDDIPTA\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df=pd.read_csv("Vaccination.csv")
df.drop(['label__decision_type_relevant','label__decision_type_sentiment_nonneutral','label__who','label__decision_type_sentiment','label__decision_type_who'],axis=1,inplace=True)

In [4]:
df['label__sentiment']=df['label__sentiment'].fillna('neutral')
df.head()

,fold,text,id,label__relevant,label__sentiment
0,train,Wow! Amazing! This is the same reason why I mi...,544637830925156000,no,neutral
1,train,@TuckOnTheRise Not vaccines. Experimental seru...,539838683352797000,yes,neutral
2,train,Cosmo Mom With a Twist of Lyme | Surviving Mom...,551847317490245000,no,neutral
3,dev,http://t.co/fjYoW5dINv: Don?t Go to Disneyland...,558106789611573000,yes,neutral
4,dev,Why does everyone in the nhl keep getting the ...,543487932645249000,no,neutral


### Break-up of Train, Dev & Test

In [5]:

train = df[df['fold'] =='train']
test = df[df['fold'] =='test']
#train=pd.concat([train,valid],axis=0)
valid = df[df['fold'] =='dev']

#Vaccination related or Not in each subgroup
print("--------Train Set----------- ")
print("Tweets from vaccination domain:{}(anti-vaccine:{}/pro-vaccine:{})".format(len(train[train['label__relevant']=='yes']),len(train[train['label__sentiment']=='negative']),len(train[train['label__relevant']=='yes'])-len(train[train['label__sentiment']=='negative'])))
print("Tweets from other domain:{}".format(len(train[train['label__relevant']!='yes'])))
print()


#Vaccination related or Not in each subgroup
print("--------Validation----------- ")
print("Tweets from vaccination domain:{}(anti-vaccine:{}/pro-vaccine:{})".format(len(valid[valid['label__relevant']=='yes']),len(valid[valid['label__sentiment']=='negative']),len(valid[valid['label__relevant']=='yes'])-len(valid[valid['label__sentiment']=='negative'])))
print("Tweets from other domain:{}".format(len(valid[valid['label__relevant']!='yes'])))
print()



#Vaccination related or Not in each subgroup
print("--------Test Set----------- ")
print("Tweets from vaccination domain:{}(anti-vaccine:{}/pro-vaccine:{})".format(len(test[test['label__relevant']=='yes']),len(test[test['label__sentiment']=='negative']),len(test[test['label__relevant']=='yes'])-len(test[test['label__sentiment']=='negative'])))
print("Tweets from other domain:{}".format(len(test[test['label__relevant']!='yes'])))
print()



--------Train Set----------- 
Tweets from vaccination domain:501(anti-vaccine:59/pro-vaccine:442)
Tweets from other domain:1378

--------Validation----------- 
Tweets from vaccination domain:252(anti-vaccine:31/pro-vaccine:221)
Tweets from other domain:701

--------Test Set----------- 
Tweets from vaccination domain:251(anti-vaccine:29/pro-vaccine:222)
Tweets from other domain:715



In [6]:
train['label__sentiment']=[0 if x=='negative' else 1 for x in train['label__sentiment'] ]
test['label__sentiment']=[0 if x=='negative' else 1 for x in test['label__sentiment'] ]
valid['label__sentiment']=[0 if x=='negative' else 1 for x in valid['label__sentiment'] ]

### Text Vectorization

In [7]:
y_train=train['label__sentiment']
y_test=test['label__sentiment']
y_valid=valid['label__sentiment']

train_text=train['text']
valid_text=valid['text']
test_text=test['text']

train.drop(['label__sentiment','id','fold','text','label__relevant'],axis=1,inplace=True)
valid.drop(['label__sentiment','id','fold','text','label__relevant'],axis=1,inplace=True)
test.drop(['label__sentiment','id','fold','text','label__relevant'],axis=1,inplace=True)



In [8]:
vec=TfidfVectorizer(stop_words='english')
x_train=vec.fit_transform(train_text)
x_valid=vec.transform(valid_text)
x_test=vec.transform(test_text)
print(" Vectorization of text complete...")
print(" x_train shape: {} and x_test shape:{}".format(x_train.shape,x_test.shape))

 Vectorization of text complete...
 x_train shape: (1879, 7179) and x_test shape:(966, 7179)


In [9]:
train_text=pd.DataFrame(x_train.toarray(),columns=list(vec.get_feature_names()))
test_text=pd.DataFrame(x_test.toarray(),columns=list(vec.get_feature_names()))
valid_text=pd.DataFrame(x_valid.toarray(),columns=list(vec.get_feature_names()))


train_data=train.as_matrix()
train_text=train_text.as_matrix()
train_d=np.concatenate((train_data,train_text), axis=1)

test_data=test.as_matrix()
test_text=test_text.as_matrix()
test_d=np.concatenate((test_data,test_text), axis=1)

valid_data=valid.as_matrix()
valid_text=valid_text.as_matrix()
valid_d=np.concatenate((valid_data,valid_text), axis=1)


In [10]:
print(train_d)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Model Development (Without Sampling)

In [11]:
from sklearn import linear_model
clf=linear_model.LogisticRegressionCV(cv=5)
clf.fit(train_d,y_train)
valid_pred=clf.predict(valid_d)
test_pred=clf.predict(test_d)


tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
print("***************   Logistic Regression *-**********************")
print("ROC-AUC score-- Validation:{} Test:{}".format(roc_auc_score(y_valid,valid_pred),roc_auc_score(y_test,test_pred)))
print("F1--- validation:{} Test:{}".format(f1_score(y_valid,valid_pred),f1_score(y_test,test_pred)))
print("FP rate--- validation:{} Test:{}".format((fpv/(fpv+tnv))*100,(fpt/(fpt+tnt))*100))




***************   Logistic Regression *-**********************
ROC-AUC score-- Validation:0.5623469316352949 Test:0.5339491406911273
F1--- validation:0.9833958221746116 Test:0.9852631578947368
FP rate--- validation:87.09677419354838 Test:93.10344827586206


In [13]:
max_depth=[2,3,5,6]
for m in max_depth:
    clf = ensemble.GradientBoostingClassifier(max_depth=m)
    clf.fit(train_d, y_train)
    
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)

    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    print("***************   Gradient (max_depth:{}) *-**********************".format(m))
    print("ROC-AUC score-- Validation:{} Test:{}".format(roc_auc_score(y_valid,valid_pred),roc_auc_score(y_test,test_pred)))
    print("F1--- validation:{} Test:{}".format(f1_score(y_valid,valid_pred),f1_score(y_test,test_pred)))
    print("FP rate--- validation:{} Test:{}".format((fpv/(fpv+tnv))*100,(fpt/(fpt+tnt))*100))
    print("\n\n")

***************   Gradient (max_depth:2) *-**********************
ROC-AUC score-- Validation:0.5279196697222028 Test:0.5140396717329703
F1--- validation:0.9801608579088471 Test:0.9820675105485233
FP rate--- validation:93.54838709677419 Test:96.55172413793103



***************   Gradient (max_depth:3) *-**********************
ROC-AUC score-- Validation:0.5757644671471556 Test:0.4973319103521878
F1--- validation:0.9811929070392261 Test:0.982086406743941
FP rate--- validation:83.87096774193549 Test:100.0



***************   Gradient (max_depth:5) *-**********************
ROC-AUC score-- Validation:0.5424218039325449 Test:0.49466382070437565
F1--- validation:0.9790435249865663 Test:0.9793977812995246
FP rate--- validation:90.32258064516128 Test:100.0



***************   Gradient (max_depth:6) *-**********************
ROC-AUC score-- Validation:0.5424218039325449 Test:0.49466382070437565
F1--- validation:0.9790435249865663 Test:0.9793977812995246
FP rate--- validation:90.32258064516128 T

In [14]:
kernel=['rbf','linear']
c_val=[1,5,10,100]
for k in kernel:
    for c in c_val:
        clf =svm.SVC(C=c,kernel=k)
        clf.fit(train_d, y_train)

        valid_pred=clf.predict(valid_d)
        test_pred=clf.predict(test_d)

        tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
        tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
        print("***************  SVM (C:{}, kernel:{}) *-**********************".format(c,k))
        print("ROC-AUC score-- Validation:{} Test:{}".format(roc_auc_score(y_valid,valid_pred),roc_auc_score(y_test,test_pred)))
        print("F1--- validation:{} Test:{}".format(f1_score(y_valid,valid_pred),f1_score(y_test,test_pred)))
        print("FP rate--- validation:{} Test:{}".format((fpv/(fpv+tnv))*100,(fpt/(fpt+tnt))*100))
        print("\n\n")
        

***************  SVM (C:1, kernel:rbf) *-**********************
ROC-AUC score-- Validation:0.5 Test:0.5
F1--- validation:0.9834666666666666 Test:0.9847609038360483
FP rate--- validation:100.0 Test:100.0



***************  SVM (C:5, kernel:rbf) *-**********************
ROC-AUC score-- Validation:0.5 Test:0.5
F1--- validation:0.9834666666666666 Test:0.9847609038360483
FP rate--- validation:100.0 Test:100.0



***************  SVM (C:10, kernel:rbf) *-**********************
ROC-AUC score-- Validation:0.5 Test:0.5
F1--- validation:0.9834666666666666 Test:0.9847609038360483
FP rate--- validation:100.0 Test:100.0



***************  SVM (C:100, kernel:rbf) *-**********************
ROC-AUC score-- Validation:0.5 Test:0.5
F1--- validation:0.9834666666666666 Test:0.9847609038360483
FP rate--- validation:100.0 Test:100.0



***************  SVM (C:1, kernel:linear) *-**********************
ROC-AUC score-- Validation:0.5 Test:0.5
F1--- validation:0.9834666666666666 Test:0.9847609038360483
FP rat

### Clearly Less Data! Does not work!

#### 1. Oversampling of minor class(anti-vacc)


In [16]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)

logistic_auc_test=[]

logistic_f1_test=[]

logistic_fp_test=[]
logistic_auc_valid=[]

logistic_f1_valid=[]

logistic_fp_valid=[]
for s in seeds:
    if(s%10==0):
        print("{} completed".format(s))
    ros=ros=RandomOverSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    
    clf=linear_model.LogisticRegressionCV()
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    logistic_auc_valid.append(roc_auc_score(y_valid,valid_pred))
    logistic_f1_valid.append(f1_score(y_valid,valid_pred))
    logistic_fp_valid.append(fpv/(fpv+tnv))
    
    logistic_auc_test.append(roc_auc_score(y_test,test_pred))
    logistic_f1_test.append(f1_score(y_test,test_pred))
    logistic_fp_test.append(fpt/(fpt+tnt))
    
print(" Average AUC score over all seeds...")
print("Validation:{}  Test:{}".format(sum(logistic_auc_valid)/len(logistic_auc_valid),sum(logistic_auc_test)/len(logistic_auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(logistic_f1_valid)/len(logistic_f1_valid),sum(logistic_f1_test)/len(logistic_f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(logistic_fp_valid)/len(logistic_fp_valid))*100,(sum(logistic_fp_test)/len(logistic_fp_test))*100))
 
        
    

0 completed
10 completed
 Average AUC score over all seeds...
Validation:0.559635434889091  Test:0.5495896662127847


 Average F1 score over all seeds...
Validation:0.9806659505907624  Test:0.9841772151898737


 Average FP rate over all seeds...
Validation:87.09677419354838  Test:89.65517241379307


In [144]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed ".format(s))
    ros=ros=RandomOverSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=2)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed 
10 completed 
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.818938810821814  Test:0.8507291663181701


 Average F1 score over all seeds...
Validation:0.919807265358399  Test:0.9300423618702294


 Average FP rate over all seeds...
Validation:21.994134897360702  Test:17.241379310344833


In [147]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed".format(s))
    ros=ros=RandomOverSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=3)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed
10 completed
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.6836915795701045  Test:0.7912182212958719


 Average F1 score over all seeds...
Validation:0.9453648573599999  Test:0.9553163553333739


 Average FP rate over all seeds...
Validation:54.54545454545454  Test:34.1692789968652


In [148]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed".format(s))
    ros=ros=RandomOverSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=5)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed
10 completed
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.6328824244120586  Test:0.6871978534842406


 Average F1 score over all seeds...
Validation:0.9640049231979503  Test:0.9661473748752126


 Average FP rate over all seeds...
Validation:68.62170087976541  Test:57.68025078369906


In [149]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed ...".format(s))
    ros=ros=RandomOverSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=6)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (6)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed ...
10 completed ...
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.6213144318420366  Test:0.6328123170393071


 Average F1 score over all seeds...
Validation:0.9711772376253268  Test:0.9713265808661272


 Average FP rate over all seeds...
Validation:72.43401759530794  Test:69.90595611285268


In [17]:
c=[1,5,10,100]
for c_val in c:
    
    seeds=[]
    for i in range(0 , 11):
        seeds.append(i)
    auc_valid=[]
    auc_test=[]
    f1_valid=[]
    f1_test=[]
    fp_valid=[]
    fp_test=[]
    for s in seeds:
        if(s%5==0):
            print("{} completed ...".format(s))
        ros=ros=RandomOverSampler(random_state=s)
        x_train,t_train=ros.fit_sample(train_d,y_train)
        clf=svm.SVC(C=c_val,kernel='rbf')
        clf.fit(x_train,t_train)
        valid_pred=clf.predict(valid_d)
        test_pred=clf.predict(test_d)
        tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
        tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()

        auc_valid.append(roc_auc_score(y_valid,valid_pred))
        f1_valid.append(f1_score(y_valid,valid_pred))
        fp_valid.append(fpv/(fpv+tnv))

        auc_test.append(roc_auc_score(y_test,test_pred))
        f1_test.append(f1_score(y_test,test_pred))
        fp_test.append(fpt/(fpt+tnt))
    print("***********   SVM RBF (C={})***************************".format(c_val))
    print(" Average AUC score over all seeds") 
    print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
    print("\n")
    print(" Average F1 score over all seeds...")
    print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
    print("\n")
    print(" Average FP rate over all seeds...")
    print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))


        



0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=1)***************************
 Average AUC score over all seeds
Validation:0.692514678659805  Test:0.7967601529593211


 Average F1 score over all seeds...
Validation:0.9487249750863946  Test:0.9546537277261506


 Average FP rate over all seeds...
Validation:53.3724340175953  Test:32.915360501567406
0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=5)***************************
 Average AUC score over all seeds
Validation:0.692514678659805  Test:0.7967601529593211


 Average F1 score over all seeds...
Validation:0.9487249750863946  Test:0.9546537277261506


 Average FP rate over all seeds...
Validation:53.3724340175953  Test:32.915360501567406
0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=10)***************************
 Average AUC score over all seeds
Validation:0.692514678659805  Test:0.7967601529593211


 Average F1 score over all seeds...
Validation:0.948724

In [18]:
c=[1,5,10,100]
for c_val in c:
    
    seeds=[]
    for i in range(0 , 11):
        seeds.append(i)
    auc_valid=[]
    auc_test=[]
    f1_valid=[]
    f1_test=[]
    fp_valid=[]
    fp_test=[]
    for s in seeds:
        if(s%10==0):
            print("{} completed ...".format(s))
        ros=ros=RandomOverSampler(random_state=s)
        x_train,t_train=ros.fit_sample(train_d,y_train)
        clf=svm.SVC(C=c_val,kernel='linear')
        clf.fit(x_train,t_train)
        valid_pred=clf.predict(valid_d)
        test_pred=clf.predict(test_d)
        tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
        tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()

        auc_valid.append(roc_auc_score(y_valid,valid_pred))
        f1_valid.append(f1_score(y_valid,valid_pred))
        fp_valid.append(fpv/(fpv+tnv))

        auc_test.append(roc_auc_score(y_test,test_pred))
        f1_test.append(f1_score(y_test,test_pred))
        fp_test.append(fpt/(fpt+tnt))
    print("***********   SVM Linear (C={})***************************".format(c_val))
    print(" Average AUC score over all seeds") 
    print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
    print("\n")
    print(" Average F1 score over all seeds...")
    print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
    print("\n")
    print(" Average FP rate over all seeds...")
    print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))


        



0 completed ...
10 completed ...
***********   SVM Linear (C=1)***************************
 Average AUC score over all seeds
Validation:0.5618046322860542  Test:0.5495896662127847


 Average F1 score over all seeds...
Validation:0.9828510182207932  Test:0.9841772151898737


 Average FP rate over all seeds...
Validation:87.09677419354838  Test:89.65517241379307
0 completed ...
10 completed ...
***********   SVM Linear (C=5)***************************
 Average AUC score over all seeds
Validation:0.5618046322860542  Test:0.5328819048320025


 Average F1 score over all seeds...
Validation:0.9828510182207932  Test:0.9841938883034771


 Average FP rate over all seeds...
Validation:87.09677419354838  Test:93.10344827586209
0 completed ...
10 completed ...
***********   SVM Linear (C=10)***************************
 Average AUC score over all seeds
Validation:0.5618046322860542  Test:0.5328819048320025


 Average F1 score over all seeds...
Validation:0.9828510182207932  Test:0.9841938883034771


### Random Under Sampling

In [19]:

seeds=[]
for i in range(0 , 11):
    seeds.append(i)

logistic_auc_test=[]

logistic_f1_test=[]

logistic_fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    
    clf=linear_model.LogisticRegressionCV()
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    logistic_auc_valid.append(roc_auc_score(y_valid,valid_pred))
    logistic_f1_valid.append(f1_score(y_valid,valid_pred))
    logistic_fp_valid.append(fpv/(fpv+tnv))
    
    logistic_auc_test.append(roc_auc_score(y_test,test_pred))
    logistic_f1_test.append(f1_score(y_test,test_pred))
    logistic_fp_test.append(fpt/(fpt+tnt))
    
print(" Average AUC score over all seeds...")
print("Validation:{}  Test:{}".format(sum(logistic_auc_valid)/len(logistic_auc_valid),sum(logistic_auc_test)/len(logistic_auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(logistic_f1_valid)/len(logistic_f1_valid),sum(logistic_f1_test)/len(logistic_f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(logistic_fp_valid)/len(logistic_fp_valid))*100,(sum(logistic_fp_test)/len(logistic_fp_test))*100))
 
        
    

0 completed
10 completed
 Average AUC score over all seeds...
Validation:0.6967751477407904  Test:0.8409985848251774


 Average F1 score over all seeds...
Validation:0.9058446428428389  Test:0.8411003651733483


 Average FP rate over all seeds...
Validation:45.89442815249268  Test:4.702194357366771


In [20]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed ".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=2)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed 
10 completed 
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.8666293471415577  Test:0.8571208719885717


 Average F1 score over all seeds...
Validation:0.8931013395476061  Test:0.8922066572215197


 Average FP rate over all seeds...
Validation:7.6246334310850425  Test:9.404388714733543


In [21]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed ".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=3)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed 
10 completed 
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.8471495092270406  Test:0.861841466964199


 Average F1 score over all seeds...
Validation:0.8953755583117805  Test:0.8959908455211728


 Average FP rate over all seeds...
Validation:12.023460410557183  Test:9.090909090909088


In [22]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed ".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=5)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed 
10 completed 
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.8381180781292741  Test:0.859273744325082


 Average F1 score over all seeds...
Validation:0.8964797497264374  Test:0.8983758132595141


 Average FP rate over all seeds...
Validation:14.076246334310849  Test:10.031347962382444


In [23]:
seeds=[]
for i in range(0 , 11):
    seeds.append(i)
auc_valid=[]
auc_test=[]
f1_valid=[]
f1_test=[]
fp_valid=[]
fp_test=[]
for s in seeds:
    if(s%10==0):
        print("{} completed ".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=6)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)


    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    auc_valid.append(roc_auc_score(y_valid,valid_pred))
    f1_valid.append(f1_score(y_valid,valid_pred))
    fp_valid.append(fpv/(fpv+tnv))
    
    auc_test.append(roc_auc_score(y_test,test_pred))
    f1_test.append(f1_score(y_test,test_pred))
    fp_test.append(fpt/(fpt+tnt))

print("***********   Gradient Boosting max_depth (2)***************************")
print(" Average AUC score over all seeds") 
print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
print("\n")
print(" Average F1 score over all seeds...")
print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
print("\n")
print(" Average FP rate over all seeds...")
print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))
 
        
    

0 completed 
10 completed 
***********   Gradient Boosting max_depth (2)***************************
 Average AUC score over all seeds
Validation:0.8315754988835948  Test:0.8554932536642322


 Average F1 score over all seeds...
Validation:0.8971826664826034  Test:0.8993366993584889


 Average FP rate over all seeds...
Validation:15.542521994134898  Test:10.971786833855797


In [24]:
c=[1,5,10,100]
for c_val in c:
    
    seeds=[]
    for i in range(0 , 11):
        seeds.append(i)
    auc_valid=[]
    auc_test=[]
    f1_valid=[]
    f1_test=[]
    fp_valid=[]
    fp_test=[]
    for s in seeds:
        if(s%5==0):
            print("{} completed ...".format(s))
        ros=ros=RandomUnderSampler(random_state=s)
        x_train,t_train=ros.fit_sample(train_d,y_train)
        clf=svm.SVC(C=c_val,kernel='rbf')
        clf.fit(x_train,t_train)
        valid_pred=clf.predict(valid_d)
        test_pred=clf.predict(test_d)
        tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
        tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()

        auc_valid.append(roc_auc_score(y_valid,valid_pred))
        f1_valid.append(f1_score(y_valid,valid_pred))
        fp_valid.append(fpv/(fpv+tnv))

        auc_test.append(roc_auc_score(y_test,test_pred))
        f1_test.append(f1_score(y_test,test_pred))
        fp_test.append(fpt/(fpt+tnt))
    print("***********   SVM RBF (C={})***************************".format(c_val))
    print(" Average AUC score over all seeds") 
    print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
    print("\n")
    print(" Average F1 score over all seeds...")
    print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
    print("\n")
    print(" Average FP rate over all seeds...")
    print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))


        



0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=1)***************************
 Average AUC score over all seeds
Validation:0.6845312688850579  Test:0.76072839683777


 Average F1 score over all seeds...
Validation:0.9522646839052541  Test:0.9567626719092917


 Average FP rate over all seeds...
Validation:55.71847507331379  Test:40.75235109717869
0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=5)***************************
 Average AUC score over all seeds
Validation:0.6845312688850579  Test:0.76072839683777


 Average F1 score over all seeds...
Validation:0.9522646839052541  Test:0.9567626719092917


 Average FP rate over all seeds...
Validation:55.71847507331379  Test:40.75235109717869
0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=10)***************************
 Average AUC score over all seeds
Validation:0.6845312688850579  Test:0.76072839683777


 Average F1 score over all seeds...
Validation:0.952264683

In [27]:
c=[1,5,10,100]
for c_val in c:
    
    seeds=[]
    for i in range(0,11):
        seeds.append(i)
    auc_valid=[]
    auc_test=[]
    f1_valid=[]
    f1_test=[]
    fp_valid=[]
    fp_test=[]
    for s in seeds:
        if(s%5==0):
            print("{} completed ...".format(s))
        ros=ros=RandomUnderSampler(random_state=s)
        x_train,t_train=ros.fit_sample(train_d,y_train)
        clf=svm.SVC(C=c_val,kernel='linear')
        clf.fit(x_train,t_train)
        valid_pred=clf.predict(valid_d)
        test_pred=clf.predict(test_d)
        tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
        tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()

        auc_valid.append(roc_auc_score(y_valid,valid_pred))
        f1_valid.append(f1_score(y_valid,valid_pred))
        fp_valid.append(fpv/(fpv+tnv))

        auc_test.append(roc_auc_score(y_test,test_pred))
        f1_test.append(f1_score(y_test,test_pred))
        fp_test.append(fpt/(fpt+tnt))
    print("***********   SVM RBF (C={})***************************".format(c_val))
    print(" Average AUC score over all seeds") 
    print("Validation:{}  Test:{}".format(sum(auc_valid)/len(auc_valid),sum(auc_test)/len(auc_test)))
    print("\n")
    print(" Average F1 score over all seeds...")
    print("Validation:{}  Test:{}".format(sum(f1_valid)/len(f1_valid),sum(f1_test)/len(f1_test)))
    print("\n")
    print(" Average FP rate over all seeds...")
    print("Validation:{}  Test:{}".format((sum(fp_valid)/len(fp_valid))*100,(sum(fp_test)/len(fp_test))*100))


        



0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=1)***************************
 Average AUC score over all seeds
Validation:0.8525025286098689  Test:0.8675004934711261


 Average F1 score over all seeds...
Validation:0.864956239323013  Test:0.8749291330034865


 Average FP rate over all seeds...
Validation:5.865102639296187  Test:4.3887147335423204
0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=5)***************************
 Average AUC score over all seeds
Validation:0.8302507617636018  Test:0.8547706112016275


 Average F1 score over all seeds...
Validation:0.8693944889676021  Test:0.8800769206278143


 Average FP rate over all seeds...
Validation:11.143695014662756  Test:7.836990595611285
0 completed ...
5 completed ...
10 completed ...
***********   SVM RBF (C=10)***************************
 Average AUC score over all seeds
Validation:0.8313353604620833  Test:0.8540764060581526


 Average F1 score over all seeds...
Validation:0.87

## Best Classifier of 3 Different Approach afte tuning on Validation Data
## Result on Test Data

##### Without Sampling

Parameter Tuned

    Logsitic Regression ( Validation False Postive: 87% / Test False Positive : 93%)
    Gradient Boosting (max depth=2)  ( Validation FP Rate :93% / Test FP Rate: 96%)
    SVM( RBF/Linear) (Validation FP Rate:100% / Test FP Rate:100%)

##### With Oversampling 

Parameter Tuned

    Logistic Regression( Validation FP: 87% / Test FP: 89%)
    Gradient Boosting( Max Depth=2)( Validation FP: 21% / Test FP:17%)
    SVM(RBF C=100) (Validation FP: 49% / Test FP: 31%)

##### With Undersampling

Parameter Tuned

    Logistic Regression( Validation FP: 45% / Test FP : 4%)
    Gradient Boosting ( Validation FP: 7% / Test FP:9%)
    SVM Linear (C=1) (Validation FP:6% / Test FP: 5%)

In [40]:
# The Best setup are Undersampling with SVM and GBM
tntl=[]
tptl=[]
fptl=[]
fntl=[]

for s in seeds:
    #if(s%5==0):
        #print("{} completed ...".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=svm.SVC(C=1,kernel='linear')
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)
    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    tntl.append(tnt)
    tptl.append(tpt)
    fptl.append(fpt)
    fntl.append(fnt)
    
print("Confusion Matrix in %")
print(" Actual Pos/Neutral:937/ Actual negative:29")
print("TP: {} TN:{} FP:{} FN:{}".format((sum(tptl)/len(tptl)).round(),(sum(tntl)/len(tntl)).round(),(sum(fptl)/len(fptl)).round(),(sum(fntl)/len(fntl)).round()))



    

Confusion Matrix in %
 Actual Pos/Neutral:937/ Actual negative:29
TP: 730.0 TN:28.0 FP:1.0 FN:207.0


In [41]:
# The Best setup are Undersampling with SVM and GBM
tntl=[]
tptl=[]
fptl=[]
fntl=[]

for s in seeds:
    #if(s%5==0):
        #print("{} completed ...".format(s))
    ros=ros=RandomUnderSampler(random_state=s)
    x_train,t_train=ros.fit_sample(train_d,y_train)
    clf=ensemble.GradientBoostingClassifier(max_depth=2)
    clf.fit(x_train,t_train)
    valid_pred=clf.predict(valid_d)
    test_pred=clf.predict(test_d)
    tnv, fpv, fnv, tpv = confusion_matrix(y_valid,valid_pred).ravel()
    tnt, fpt, fnt, tpt = confusion_matrix(y_test,test_pred).ravel()
    
    tntl.append(tnt)
    tptl.append(tpt)
    fptl.append(fpt)
    fntl.append(fnt)
print("Confusion Matrix in ")
print("Actual Pos/Neutral:937 Actual Neg: 29")
print("TP: {} TN:{} FP:{} FN:{}".format((sum(tptl)/len(tptl)).round(),(sum(tntl)/len(tntl)).round(),(sum(fptl)/len(fptl)).round(),(sum(fntl)/len(fntl)).round()))



    

Confusion Matrix in 
Actual Pos/Neutral:937 Actual Neg: 29
TP: 758.0 TN:26.0 FP:3.0 FN:179.0


##### NLTK Sentiment analyzer Matching and confusion matrix

In [42]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [44]:
test_=list(test_d)
print(len(test_))

966


In [48]:
test = df[df['fold'] =='test']

In [50]:

sentiment_score=[]
pos_sent=[]
neg_sent=[]
sent_label=[]
for line in test['text']:
    
    
    ss = sid.polarity_scores(line)
    #print(max(ss))
    #print(line)
    #print(ss)
    score=ss['compound']
    
    #print(ss['compound'])
    sentiment_score.append(score)
    if score>=0:
        pos_sent.append(score)
        sent_label.append(1)
    if score<0:
        neg_sent.append(score)
        sent_label.append(0)
    
            
print("{} sentiments added to the sentiment_score list".format(len(sentiment_score)))        
       

966 sentiments added to the sentiment_score list


In [54]:
tnt, fpt, fnt, tpt = confusion_matrix(y_test,sent_label).ravel()
print("AUC:{}".format(roc_auc_score(y_test,sent_label)))
print("F1:{}".format(f1_score(y_test,sent_label)))
print("Actual Pos/Neutral:937 Actual Neg: 29")
print("TP:{} TN:{} FP:{} FN:{}".format(tpt,tnt,fpt,fnt))

AUC:0.6835093659146948
F1:0.8265179677819083
Actual Pos/Neutral:937 Actual Neg: 29
TP:667 TN:19 FP:10 FN:270
